In [9]:
import os
import nibabel as nib
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

IMAGES_TO_LOAD = 10

def preprocess_nifti_image(img, max_dim = 100):
    height, width, depth = img.shape
    start = max(0, depth // 2 - max_dim // 2)
    end = min(depth, depth // 2 + max_dim // 2)
    new_img = np.zeros((height, width, max_dim), dtype=img.dtype)
    # Avoid odd dimensions during the redimension
    if (end - start) % 2 != 0:
        start += 1
    new_img[:, :, (max_dim // 2 - (end - start)//2):(max_dim // 2 + (end - start)//2)] = img[:, :, start:end]

    return new_img

def reshape_input(volume):
    # Change from [100, 512, 512, 1] to [T, H, W, C]
    reshaped = np.moveaxis(volume, -1, 0)
    reshaped = np.expand_dims(reshaped, -1) 
    return reshaped

def load_nii_images(folder, labels_file):
    preprocessed_images = []
    labels = []
    labels_df = pd.read_csv(labels_file)
    for i, file_name in enumerate(os.listdir(folder)):
        if i >= IMAGES_TO_LOAD:
            break
        if file_name.endswith('.nii.gz'):
            id = file_name.split("/")[-1].split(".")[0]
            
            file_path = os.path.join(folder, file_name)
            image_nifti = nib.load(file_path)
            image_array = image_nifti.get_fdata()
            preprocessed_image = preprocess_nifti_image(image_array)
            preprocessed_image_reshaped = reshape_input(preprocessed_image)
            preprocessed_images.append(
                preprocessed_image_reshaped
            )
            
            labels.append(
                int(
                    labels_df.loc[
                        labels_df.id == id,
                        "label"
                    ].iloc[
                        0
                    ]
                )
            )
    images_tensor = torch.tensor(preprocessed_images, dtype = torch.float32)
    # Let's repeat the input to adapt it to the 3 required channels
    images_tensor = images_tensor.repeat(1, 1, 1, 1, 3) 
    # Permutate the input to match the dimensions in the model
    images_tensor = images_tensor.permute(0, 4, 1, 2, 3)
    labels_tensor = torch.tensor(labels, dtype=torch.int64)
    return {
        "images": images_tensor, 
        "labels": labels_tensor
    }
# Load the images and reshape if necessary
dict_images = load_nii_images('../aocr2024/1_Train,Valid_Image/', "../aocr2024/TrainValid_ground_truth.csv")

TypeError: repeat() takes at most 2 arguments (5 given)

In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np

class ModifiedR3D18(nn.Module):
    def __init__(self):
        super(ModifiedR3D18, self).__init__()
        self.r3d_18 = models.video.r3d_18(pretrained=True)
        num_ftrs = self.r3d_18.fc.in_features
        self.r3d_18.fc = nn.Sequential(
            nn.Linear(num_ftrs, 1), 
            nn.Sigmoid()            
        )

    def forward(self, x):
        return self.r3d_18(x)

model = ModifiedR3D18()
model.train()  

/Users/b.freire/Documents/GitHub/accute_appendix/accute_appendix/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/b.freire/Documents/GitHub/accute_appendix/accute_appendix/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ModifiedR3D18(
  (r3d_18): VideoResNet(
    (stem): BasicStem(
      (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (conv2): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (relu): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=(3,

In [8]:
dataset = TensorDataset(dict_images["images"], dict_images["labels"])
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [5]:
num_epochs = 10
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(num_epochs):  
    for data in dataloader:    
        inputs, labels = data    

        optimizer.zero_grad()    
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


RuntimeError: Given groups=1, weight of size [64, 3, 3, 7, 7], expected input[10, 100, 512, 512, 1] to have 3 channels, but got 100 channels instead

In [2]:
dict_images["images"].shape

torch.Size([10, 100, 512, 512, 1])